In [1]:
from tti_library import ttiPsu

In [2]:
ip = '192.168.196.32'
tti = ttiPsu(ip,1)

In [3]:
V_start = tti.readOutputVolts()
status = tti.getOutputIsEnabled()
print("The current voltage on the TTI is: ", V_start, "\t V")
print("Currently the output is: ", status)

OSError: [Errno 113] No route to host

### POWER CONFIGURATION

In [10]:
def ramp_up(ip,V_start,V0):
    status = True
    tti = ttiPsu(ip,1)

    #   Makes sure output is turned off 
    try:
        tti.setOutputEnable(False)
    except:
        failure(ip)
        return False
    
    #   Turns output on
    try:
        tti.setOutputEnable(True)
    except:
        failure(ip)
        return False
    
    #	Test to ensure we can read voltages
    try:
        print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
    except:
        failure(ip)
        return False
    
    #   Sets max current to 7.5 mA
    tti.setMaxAmps(0.0075)
    tti.setTripAmps(0.01) 
    tti.setTripVolts(110) 
    #   Sets voltage step size
    tti.setStepSizeVolts(1)
    #   Begins ramping up voltage
    if V0 > 6:
        print("starting ramp")
        #   Slow at the beginning
        while tti.readOutputVolts() < 6:
            #   Stops if incrimentation fails
            try:
                tti.incrementVoltage()
                print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
                #write()
            except:
                failure(ip)
                status = False
                break
                
        #   Breaks ends the function if there was an issue in incrimentation
        if status == False:
            failure(ip)
            return False
        
        #   Speeds up incrimentation
        fast_step = 6
        tti.setStepSizeVolts(fast_step)
        
        #   Same as above but now to V0
        while tti.readOutputVolts() < V0-fast_step:
                try:    
                    tti.incrementVoltage()
                    print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
                    #write()
                except:
                    failure(ip)
                    status = False
                    break
                if status == False:
                    failure(ip)
                    return False
        
        tti.setMaxVolts(V0)
        print("The current Voltage is: " ,tti.readOutputVolts() ,"V")

    #   If voltage is lower than threshold to speed up then just do normally
    elif V0 < 7: 
        while tti.readOutputVolts() < 7:
            try:
                tti.incrementVoltage()
                print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
            except: 
                failure(ip)
                break

    tti.setStepSizeVolts(0)
    tti.setMaxVolts(V0)            
    return True

def ramp_down(ip):
    tti = ttiPsu(ip,1)
        
    #   Prints the voltage it starts at
    print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
    #write() 
    #   initializes the voltage step size to 1
    tti.setStepSizeVolts(6)
    
    #   Lowers the voltage (reverse of before)
    while tti.readOutputVolts() > 0.1:
        if tti.readOutputVolts() < 7:
            try:
                tti.setStepSizeVolts(1)
            except:
                failure(ip)        
        try:
            tti.decrementVoltage()
            print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
        except:
            failure(ip)
        
    #   Sets voltage to zero and turns off output
    tti.setMaxVolts(0)
    print("The current Voltage is: " ,tti.readOutputVolts() ,"V")
    tti.setOutputEnable(False)
  
#   Ramp voltage down to zero and turn off
def failure(ip):
    log = open("Historical.log", "a")
    log.write("ENTERING FAILURE MODE!\n")
    #write()
    tti = ttiPsu(ip,1)
    ramp_down(ip)
    tti.setOutputEnable(False)
    log.write("Failure Protocol Succesfully executed")
    log.close()

In [11]:
ramp_up(ip,0,100)

The current Voltage is:  0.01 V
starting ramp
The current Voltage is:  1.01 V
The current Voltage is:  2.01 V
The current Voltage is:  3.01 V
The current Voltage is:  4.01 V
The current Voltage is:  5.01 V
The current Voltage is:  6.01 V
The current Voltage is:  12.02 V
The current Voltage is:  18.02 V
The current Voltage is:  24.02 V
The current Voltage is:  30.02 V
The current Voltage is:  36.02 V
The current Voltage is:  42.02 V
The current Voltage is:  48.02 V
The current Voltage is:  54.02 V
The current Voltage is:  60.02 V
The current Voltage is:  66.02 V
The current Voltage is:  72.03 V
The current Voltage is:  78.03 V
The current Voltage is:  84.03 V
The current Voltage is:  90.03 V
The current Voltage is:  96.04 V
The current Voltage is:  100.04 V


True

In [12]:
V_start = tti.readOutputVolts()
status = tti.getOutputIsEnabled()
print("The current voltage on the TTI is: ", V_start, "\t V")
print("Currently the output is: ", status)

The current voltage on the TTI is:  100.04 	 V
Currently the output is:  True


In [13]:
ramp_down(ip)

The current Voltage is:  100.04 V
The current Voltage is:  94.04 V
The current Voltage is:  88.04 V
The current Voltage is:  82.04 V
The current Voltage is:  76.03 V
The current Voltage is:  70.03 V
The current Voltage is:  64.02 V
The current Voltage is:  58.03 V
The current Voltage is:  52.02 V
The current Voltage is:  46.02 V
The current Voltage is:  40.02 V
The current Voltage is:  34.02 V
The current Voltage is:  28.03 V
The current Voltage is:  22.02 V
The current Voltage is:  16.02 V
The current Voltage is:  10.01 V
The current Voltage is:  4.01 V
The current Voltage is:  3.01 V
The current Voltage is:  2.01 V
The current Voltage is:  1.01 V
The current Voltage is:  0.01 V
The current Voltage is:  0.01 V


In [14]:
V_start = tti.readOutputVolts()
status = tti.getOutputIsEnabled()
print("The current voltage on the TTI is: ", V_start, "\t V")
print("Currently the output is: ", status)

The current voltage on the TTI is:  0.0 	 V
Currently the output is:  False


### OUTPUT CONFIGURATION

In [16]:
# SET OUTPUT TRUE
tti.setOutputEnable(True)
status = tti.getOutputIsEnabled()
print("Currently the output is: ", status)

Currently the output is:  True


### USING CLASS

In [1]:
from CLASSES.TTI_library import TTI
from CLASSES.dictionary import classes_dictionary
import json
with open('CONFIG/modules_units.json', "r") as json_file:
    moduleDB = json.load(json_file)

/home/acd/rvizarr/Mx2_SlowControlsDisplay/myenv/lib64/python3.6/site-packages/paramiko/transport.py:32: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.hazmat.backends import default_backend


In [2]:
# GET LIST OF CONNECTED UNITS
PSU_list = []
# Same process will be performed for each module within the array
for module in moduleDB.keys():
    units = moduleDB[module].keys()
    for unit in units:
        kind = moduleDB[module][unit]["class"]
        object = classes_dictionary[kind]
        PSU_list.append(object(module, unit, moduleDB[module][unit]))

In [3]:
tti = PSU_list[1]
V_start = tti.readOutputVolts(1)
status = tti.getOutputIsEnabled(1)
print("The current voltage on the TTI is: ", V_start, "\t V")
print("Currently the output is: ", status)

The current voltage on the TTI is:  0.0 	 V
Currently the output is:  False


In [4]:
tti.ramp_up(100,1)

The current Voltage is:  0.01 V
Starting Ramp
The current Voltage is:  1.01 V
The current Voltage is:  2.0 V
The current Voltage is:  3.0 V
The current Voltage is:  4.01 V
The current Voltage is:  5.0 V
The current Voltage is:  6.01 V
The current Voltage is:  12.01 V
The current Voltage is:  18.01 V
The current Voltage is:  24.02 V
The current Voltage is:  30.02 V
The current Voltage is:  36.02 V
The current Voltage is:  42.02 V
The current Voltage is:  48.02 V
The current Voltage is:  54.02 V
The current Voltage is:  60.03 V
The current Voltage is:  66.02 V
The current Voltage is:  72.03 V
The current Voltage is:  78.03 V
The current Voltage is:  84.03 V
The current Voltage is:  90.04 V
The current Voltage is:  96.05 V
The current Voltage is:  96.04 V


True

In [4]:
tti.measuring_status

{'voltage': False}

In [6]:
tti.ramp_down(1)

The current Voltage is:  96.04 V
The current Voltage is:  90.04 V
The current Voltage is:  84.05 V
The current Voltage is:  78.04 V
The current Voltage is:  72.04 V
The current Voltage is:  66.03 V
The current Voltage is:  60.04 V
The current Voltage is:  54.02 V
The current Voltage is:  48.03 V
The current Voltage is:  42.02 V
The current Voltage is:  36.03 V
The current Voltage is:  30.03 V
The current Voltage is:  24.03 V
The current Voltage is:  18.02 V
The current Voltage is:  12.02 V
The current Voltage is:  6.02 V
The current Voltage is:  5.01 V
The current Voltage is:  4.02 V
The current Voltage is:  3.02 V
The current Voltage is:  2.01 V
The current Voltage is:  1.01 V
The current Voltage is:  0.02 V
The current Voltage is:  0.02 V


In [4]:
tti.measuring_status

{'voltage': False}

In [6]:
tti.CONTINUOUS_monitoring("voltage")

The current Voltage is:  96.06 V
Starting Ramp
The current Voltage is:  96.06 V
Continuous DAQ Activated: voltage. Taking data in real time


KeyboardInterrupt: 

In [8]:
tti.readOutputVolts(1)

96.06

In [10]:
tti.crate_status = False